In [ ]:
import pandas as pd
import numpy as np

### 1.Split prepared data from Milestone 1 into training and testing

#### Read and merge data

I used Google labs, so below codes are for reading the files from Google drive.

In [ ]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144556 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
# Generate auth tokens for Colab

from google.colab import auth 
auth.authenticate_user()

In [ ]:
# Generate creds for the Drive FUSE library.

from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default()
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
# Create a directory and mount Google Drive using that directory.

!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
# print ('Files in Drive:')
# !ls drive/Colab_Notebooks/M03

In [ ]:
#read two data files
df = pd.read_excel('drive/Colab_Notebooks/M03/secom.xlsx',header= None)
label = pd.read_excel('drive/Colab_Notebooks/M03/secom_labels.xlsx', header = None)

In [ ]:
#merge two data sets
label.columns = ['class', 'time']
df = df.add_prefix('sensor_')
data = pd.concat([df,label], axis=1)
data.head()

,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,...,sensor_552,sensor_553,sensor_554,sensor_555,sensor_556,sensor_557,sensor_558,sensor_559,sensor_560,sensor_561,sensor_562,sensor_563,sensor_564,sensor_565,sensor_566,sensor_567,sensor_568,sensor_569,sensor_570,sensor_571,sensor_572,sensor_573,sensor_574,sensor_575,sensor_576,sensor_577,sensor_578,sensor_579,sensor_580,sensor_581,sensor_582,sensor_583,sensor_584,sensor_585,sensor_586,sensor_587,sensor_588,sensor_589,class,time
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,-0.0034,0.9455,202.4396,0.0,7.9558,414.8710,10.0433,0.9680,192.3963,12.5190,1.4026,-5419.00,2916.50,-4043.75,751.00,0.8955,1.7730,3.0490,64.2333,2.0222,0.1632,3.5191,83.3971,9.5126,50.6170,64.2588,49.3830,66.3141,86.9555,117.5132,...,0.1827,5.7349,0.3363,39.8842,3.2687,1.0297,1.0344,0.4385,0.1039,42.3877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,533.8500,2.1113,8.95,0.3157,3.0624,0.1026,1.6765,14.9509,NaN,NaN,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1,19/07/2008 11:55:00
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,-0.0148,0.9627,200.5470,0.0,10.1548,414.7347,9.2599,0.9701,191.2872,12.4608,1.3825,-5441.50,2604.25,-3498.75,-1640.25,1.2973,2.0143,7.3900,68.4222,2.2667,0.2102,3.4171,84.9052,9.7997,50.6596,64.2828,49.3404,64.9193,87.5241,118.1188,...,0.2829,7.1196,0.4989,53.1836,3.9139,1.7819,0.9634,0.1745,0.0375,18.1087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,535.0164,2.4335,5.92,0.2653,2.0111,0.0772,1.1065,10.9003,0.0096,0.0201,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1,19/07/2008 12:32:00
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,0.0013,0.9615,202.0179,0.0,9.5157,416.7075,9.3144,0.9674,192.7035,12.5404,1.4123,-5447.75,2701.75,-4047.00,-1916.50,1.3122,2.0295,7.5788,67.1333,2.3333,0.1734,3.5986,84.7569,8.6590,50.1530,64.1114,49.8470,65.8389,84.7327,118.6128,...,0.0857,7.1619,0.3752,23.0713,3.9306,1.1386,1.5021,0.3718,0.1233,24.7524,267.064,0.9032,1.10,0.6219,0.4122,0.2562,0.4119,68.8489,535.0245,2.0293,11.21,0.1882,4.0923,0.0640,2.0952,9.2721,0.0584,0.0484,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1,19/07/2008 13:17:00
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,-0.0033,0.9629,201.8482,0.0,9.6052,422.2894,9.6924,0.9687,192.1557,12.4782,1.4011,-5468.25,2648.25,-4515.00,-1657.25,1.3137,2.0038,7.3145,62.9333,2.6444,0.2071,3.3813,84.9105,8.6789,50.5100,64.1125,49.4900,65.1951,86.6867,117.0442,...,0.6812,56.9303,17.4781,161.4081,35.3198,54.2917,1.1613,0.7288,0.2710,62.7572,268.228,0.6511,7.32,0.1630,3.5611,0.0670,2.7290,25.0363,530.5682,2.0253,9.33,0.1738,2.8971,0.0525,1.7585,8.5831,0.0202,0.0149,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1,19/07/2008 14:43:00
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,-0.0072,0.9569,201.9424,0.0,10.5661,420.5925,10.3387,0.9735,191.6037,12.4735,1.3888,-5476.25,2635.25,-3987.50,117.00,1.2887,1.9912,7.2748,62.8333,3.1556,0.2696,3.2728,86.3269,8.7677,50.2480,64.1511,49.7520,66.1542,86.1468,121.4364,...,0.4287,9.7608,0.8311,70.9706,4.9086,2.5014,0.9778,0.2156,0.0461,22.0500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,532.0155,2.0275,8.83,0.2224,3.1776,0.0706,1.6597,10.9698,NaN,NaN,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1,19/07/2008 15:22:00


#### Clean and prepare data

Clean dataset, impute missing values. 

First, check whether target attribute has NaN, if yes, drop NaN in class, since imputing class label is not that reasonable.

In [ ]:
print(sum(data['class'].isnull()))

0


Target varibles have no NaN. 

Check whether time_stamp has NaN. Again, this one is not reasonable to impute values. 

In [ ]:
print(sum(data['time'].isnull()))

0


No missing values in time_stamp.

Next step, we can loop through each column, and impute missing values with median.

In [ ]:
for i in range(0,590):
    name = 'sensor_'+str(i) 
    HasNan =data[name].isnull()
    if sum(HasNan) > 0:
        data.loc[HasNan, name] =  np.mean(data[name])

Data is cleaned with missing values.

There is no more missing values.  Let's take care of time_stamp data. Time_stamp data is a little bit messy, has different kind of format

In [ ]:
#convert time_stamp to datetime format
data['time'] = pd.to_datetime(data['time'])

Now, the data is cleaned and next, normalizing the dataset.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

data_notime = data.drop(columns='time') #exclude time when normalizing
scaler = MinMaxScaler()
nor = scaler.fit_transform(data_notime)
data_notime_nor = pd.DataFrame(nor, columns = data_notime.columns)

In [ ]:
data_notime_nor.head()

,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,...,sensor_551,sensor_552,sensor_553,sensor_554,sensor_555,sensor_556,sensor_557,sensor_558,sensor_559,sensor_560,sensor_561,sensor_562,sensor_563,sensor_564,sensor_565,sensor_566,sensor_567,sensor_568,sensor_569,sensor_570,sensor_571,sensor_572,sensor_573,sensor_574,sensor_575,sensor_576,sensor_577,sensor_578,sensor_579,sensor_580,sensor_581,sensor_582,sensor_583,sensor_584,sensor_585,sensor_586,sensor_587,sensor_588,sensor_589,class
0,0.469231,0.589292,0.499096,0.379841,0.000609,0.0,0.328562,0.965785,0.665019,0.542479,0.358362,0.880692,0.226185,0.0,0.329909,0.165668,0.056644,0.958559,0.500188,0.848739,0.814395,0.242124,0.797675,0.481224,0.538057,0.647599,0.863698,0.398104,0.261259,0.476550,0.514343,0.529986,0.009554,0.121293,0.078721,0.019008,0.921279,0.458921,0.603022,0.267583,...,0.016832,0.060752,0.054464,0.016315,0.117357,0.052390,0.015983,0.229765,0.367205,0.201402,0.374432,0.295779,0.377041,0.173204,0.184777,0.161644,0.179466,0.168219,0.219091,0.795609,0.642926,0.011995,0.116901,0.011971,0.150986,0.011285,0.112244,0.321759,0.177698,0.159274,0.132828,0.709375,0.012325,0.017510,0.011880,0.320455,0.173076,0.155193,0.135182,0.0
1,0.575003,0.445535,0.666763,0.393982,0.000133,0.0,0.428942,0.969673,0.656639,0.412315,0.228669,0.932908,0.205145,0.0,0.457039,0.165391,0.048683,0.963740,0.476296,0.830040,0.740957,0.238978,0.712274,0.525355,0.455345,0.938169,0.981245,0.964903,0.487686,0.562509,0.701594,0.492686,0.076817,0.139530,0.083009,0.019793,0.916991,0.000000,0.757277,0.295521,...,0.030859,0.097870,0.080038,0.025645,0.161878,0.071466,0.029880,0.115285,0.104204,0.046262,0.115808,0.295779,0.377041,0.173204,0.184777,0.161644,0.179466,0.168219,0.219091,0.799892,0.826067,0.005277,0.093239,0.005750,0.102807,0.004934,0.068396,0.221387,0.220339,0.181159,0.282386,0.753125,0.034637,0.036965,0.033099,0.221387,0.220339,0.181159,0.282386,0.0
2,0.308868,0.583388,0.493903,0.457065,0.000744,0.0,0.283455,0.965008,0.542759,0.448168,0.411832,0.929265,0.221497,0.0,0.420091,0.169405,0.049237,0.957079,0.506806,0.855614,0.849836,0.238104,0.738940,0.480961,0.445790,0.948944,0.988650,0.989554,0.418016,0.585923,0.554980,0.559058,0.070202,0.067069,0.032024,0.014189,0.967976,0.302570,0.000000,0.318310,...,0.018618,0.024819,0.080819,0.018547,0.061075,0.071960,0.017995,0.983876,0.300757,0.246729,0.186578,0.358488,0.601972,0.004113,0.899070,0.000000,0.869764,0.003337,0.804376,0.799922,0.596317,0.017006,0.057042,0.018066,0.077769,0.015949,0.050771,0.629073,0.589309,0.500000,0.112383,0.562500,0.020612,0.021401,0.020152,0.629073,0.589309,0.500000,0.112383,1.0
3,0.400385,0.466998,0.543479,0.244894,0.000574,0.0,0.469123,0.946345,0.638590,0.319564,0.359499,0.933515,0.219610,0.0,0.425265,0.180762,0.053078,0.960286,0.495005,0.835631,0.808915,0.235237,0.724308,0.443066,0.454757,0.950029,0.976130,0.955045,0.190989,0.695296,0.689243,0.479595,0.077053,0.068333,0.067953,0.014225,0.932047,0.090745,0.530100,0.245947,...,1.000000,0.245416,1.000000,1.000000,0.524168,1.000000,1.000000,0.434376,0.656406,0.591822,0.591411,0.375329,0.348325,0.200886,0.210858,0.231504,0.203801,0.190569,0.267141,0.783557,0.594043,0.012838,0.050282,0.010993,0.055956,0.012198,0.043312,0.309942,0.152542,0.123188,0.100153,0.662500,0.009137,0.007782,0.008734,0.309942,0.152542,0.123188,0.100153,0.0
4,0.471367,0.500400,0.678327,0.357067,0.000765,0.0,0.387631,0.960342,0.670606,0.392050,0.315131,0.915301,0.220658,0.0,0.480818,0.177310,0.059646,0.972126,0.483114,0.834120,0.763975,0.234118,0.720752,0.485779,0.516127,0.931950,0.969992,0.949862,0.185584,0.875018,0.938247,0.439918,0.140226,0.073974,0.041585,0.015487,0.958415,0

#### Handle class imbalance problem

In [ ]:
# import SMOTE package
from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# handle imbalance data set using SMOTE
X = data_notime_nor.iloc[:,:-2]
y= data_notime_nor['class']

#split data first, then apply SMOTE
X_train, X_test, y_train, y_test = train_test_split (X,y, random_state =0)

print('Original dataset shape {}'.format(Counter(y)))
sm = SMOTE(random_state=0)
X_res, y_res = sm.fit_sample(X_train, y_train)
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0.0: 1463, 1.0: 104})
Resampled dataset shape Counter({0.0: 1092, 1.0: 1092})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


### 1. Build a simple neural network and DNN model (Code from L08)

In [ ]:
from random import random
from math import exp
from random import randrange

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network  


# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
#         print(weights[i])
#         print (inputs[i])
#         print(type(weights[i]), type( inputs[i]))
        activation += weights[i] * inputs[i]
        
    return activation

# Transfer neuron activation (activation function)
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
#     print('row:', row)
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
#     print(inputs)
    return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
#         print('network', network)
        layer = network[i]
#         print ('layer', layer)
        errors = list()
        if i != len(network)-1:
#             print(len(network))
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
#                     print('network', network)
#                     print("neuron",neuron)
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
                
        for j in range(len(layer)):
#             print(layer)
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
 
# 
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.4f, error=%.3f' % (epoch, l_rate, sum_error))
  
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [ ]:
data_nor = data_notime_nor.values.tolist()

### 1. Build a simple neural network

In [ ]:
# evaluate algorithm with a simple model with 1 hidden layers
n_folds = 5
l_rate = 0.001
n_epoch = 5
n_hidden = 1
scores = evaluate_algorithm(data_nor, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/(len(scores))))

>epoch=0, lrate=0.001, error=874.057
>epoch=1, lrate=0.001, error=758.691
>epoch=2, lrate=0.001, error=638.182
>epoch=3, lrate=0.001, error=525.967
>epoch=4, lrate=0.001, error=433.878
>epoch=0, lrate=0.001, error=483.942
>epoch=1, lrate=0.001, error=400.968
>epoch=2, lrate=0.001, error=342.445
>epoch=3, lrate=0.001, error=301.210
>epoch=4, lrate=0.001, error=271.691
>epoch=0, lrate=0.001, error=513.090
>epoch=1, lrate=0.001, error=421.594
>epoch=2, lrate=0.001, error=354.217
>epoch=3, lrate=0.001, error=306.534
>epoch=4, lrate=0.001, error=272.923
>epoch=0, lrate=0.001, error=721.498
>epoch=1, lrate=0.001, error=612.049
>epoch=2, lrate=0.001, error=505.217
>epoch=3, lrate=0.001, error=414.900
>epoch=4, lrate=0.001, error=346.714
>epoch=0, lrate=0.001, error=441.038
>epoch=1, lrate=0.001, error=367.637
>epoch=2, lrate=0.001, error=315.043
>epoch=3, lrate=0.001, error=278.272
>epoch=4, lrate=0.001, error=252.436
Scores: [94.24920127795528, 92.97124600638978, 92.33226837060703, 93.610223

In this one hidden layer neural neworks, has high mean accuracy to be 93.42%.

### 2. Build a DNN 

In [ ]:
# evaluate algorithm with DNN with 5 hidden layers and 10 epoch
n_folds = 5
l_rate = 0.001
n_epoch = 10
n_hidden = 5
scores = evaluate_algorithm(data_nor, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/(len(scores))))

>epoch=0, lrate=0.0010, error=970.241
>epoch=1, lrate=0.0010, error=746.285
>epoch=2, lrate=0.0010, error=440.382
>epoch=3, lrate=0.0010, error=266.251
>epoch=4, lrate=0.0010, error=204.126
>epoch=5, lrate=0.0010, error=179.457
>epoch=6, lrate=0.0010, error=167.528
>epoch=7, lrate=0.0010, error=160.862
>epoch=8, lrate=0.0010, error=156.744
>epoch=9, lrate=0.0010, error=154.012
>epoch=0, lrate=0.0010, error=1180.322
>epoch=1, lrate=0.0010, error=1165.283
>epoch=2, lrate=0.0010, error=1142.359
>epoch=3, lrate=0.0010, error=1104.029
>epoch=4, lrate=0.0010, error=1031.324
>epoch=5, lrate=0.0010, error=875.197
>epoch=6, lrate=0.0010, error=587.900
>epoch=7, lrate=0.0010, error=336.957
>epoch=8, lrate=0.0010, error=234.506
>epoch=9, lrate=0.0010, error=197.292
>epoch=0, lrate=0.0010, error=1114.095
>epoch=1, lrate=0.0010, error=1053.765
>epoch=2, lrate=0.0010, error=928.047
>epoch=3, lrate=0.0010, error=675.901
>epoch=4, lrate=0.0010, error=398.061
>epoch=5, lrate=0.0010, error=265.760
>epoc

In this 5 hidden layer neural neworks, has high mean accuracy to be 93.35%.

### 3.Build a RNN model

In [ ]:
import keras

In [ ]:
print('# of Training Samples: {}'.format(len(X_res)))
print('# of Test Samples: {}'.format(len(X_test)))

num_classes = max(y_res) + 1
print('# of Classes: {}'.format(num_classes))

# of Training Samples: 2184
# of Test Samples: 392
# of Classes: 2.0


In [ ]:
print(X_res.shape)
print(y_res.shape)

(2184, 589)
(2184,)


In [ ]:
#construct model
import tensorflow as tf
from tensorflow import keras
# Construct our model
in_length = 589
model = tf.keras.Sequential()
model.add(keras.layers.Embedding(10000, 32, input_length= in_length))
model.add(keras.layers.LSTM(64,activation="tanh")) #with one LSTM layer
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_res, y_res, validation_split=0.2, epochs=3, batch_size=3)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 589, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 344,897
Trainable params: 344,897
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
583/583 [==============================] - 157s 269ms/step - loss: 0.6672 - accuracy: 0.6251 - val_loss: 0.9197 - val_accuracy: 0.0000e+00
Epoch 2/3
583/583 [==============================] - 160s 274ms/step - loss: 0.6638 - accuracy: 0.6251 - val_loss: 1.0310 - val_accuracy: 0.0000e+00
Epoch 3/3
583/583 [==============================] - 158s 271ms/step - loss: 

In [ ]:
# Evaluate model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 94.64%


#### Summarize your findings with examples.  Explain what the manufacturer should focus on to optimize the diaper manufacturing process.

1. All models, simple neural networks, DNN and RNN, have very good performance with high accuracy. Among them, RNN has the highest accuracy with one LSTM layer.
2. I used LSTM, a type of RNN, is capable of learning order depence in sequence prediction problems. In this work, we found that LSTM gives the best model, suggesting that the order the sensor detections matter. 

To optimize the diaper manufacturing process, we can print out the prediction for every a hundred sensors. If it is a bad diaper, we can give up that diaper for the rest of sensor detections since LSTM predicts well on sequencing data. 

### Feedback
Neural network is the most challenging part in this course. Tuning hyperparameters is tricky, and don't which directions I should go. Neural network gives good predictions though. 
1. In this work, I didn't use timestamp for trainning. How do I include timestamp in neural network?
